In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2

In [ ]:
color_lena_img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/cv_2024_1/lena.bmp', cv2.IMREAD_COLOR)
color_lena_img = cv2.cvtColor(color_lena_img, cv2.COLOR_BGR2RGB) # Convert BGR to RGB

plt.imshow(color_lena_img)
plt.show()

In [ ]:
# prompt:  Cluster the RGB image using k-means method. (Use kmeans() functions in sklearn module)
from sklearn.cluster import KMeans

# 이미지의 모든 채널의 픽셀을 2차원으로 변환rgb
pixels = color_lena_img.reshape((-1, 3)) # rgb이므로 3으로 설정

# KMeans clustering 함수 적용
kmeans = KMeans(n_clusters=3)  # rgb이므로 3개의 군집으로 설정
kmeans.fit(pixels) # 군집화 진행

# 각 픽셀이 어느 군집에 속하는지 레이블 지정
labels = kmeans.labels_

# 각 군집의 대표 rgb색을 가져옴(3x3)
cluster_centers = kmeans.cluster_centers_

# 각 픽셀을 해당 클러스터 중심 색상으로 대체
# cluster_centers[labels]는 모든 픽셀의 RGB를 클러스터 색으로 바꾼 배열
segmented_img = cluster_centers[labels].reshape(color_lena_img.shape) # 원래 이미지의 shape (높이, 너비, 3)로 다시 reshape
segmented_img = segmented_img.astype(np.uint8) # 0~255사이의 픽셀값으로 보정

# Display the segmented image
plt.imshow(segmented_img)
plt.show()


In [ ]:
# k-means 교수님 버전


In [ ]:
# prompt: # prompt:  Design the Harris corner detector.
# # a) Obtain the feature possibility map using Harris corner detector.
# # b) Obtain the local maximum key points.
def gaussian_kernel(size, sigma=1):
    size = int(size) // 2
    x, y = np.mgrid[-size:size+1, -size:size+1]
    normal = 1 / (2.0 * np.pi * sigma**2)
    g =  np.exp(-((x**2 + y**2) / (2.0*sigma**2))) * normal
    return g

def convolve2D(image, kernel, padding=0, strides=1):
    kernel_height, kernel_width = kernel.shape
    image_height, image_width = image.shape
    output_height = (image_height - kernel_height + 2 * padding) // strides + 1
    output_width  = (image_width  - kernel_width  + 2 * padding) // strides + 1
    output = np.zeros((output_height, output_width))
    for y in range(output_height):
        for x in range(output_width):
            output[y,x] = np.sum(image[y*strides:y*strides+kernel_height,x*strides:x*strides+kernel_width] * kernel)
    return output

In [ ]:
def harris_corner_detector(gray_img, ksize=3, k=0.04, threshold=0.01):
    # 소벨 필터링으로 이미지 x,y방향 기울기 계산
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    Ix = convolve2D(gray_img, sobel_x)
    Iy = convolve2D(gray_img, sobel_y)

    # 기울기를 이용하여 미분 계산 (M 행렬 계산)
    Ix2 = Ix * Ix
    Iy2 = Iy * Iy
    Ixy = Ix * Iy

    # 가우시안 필터링 적용
    gaussian_kernel_size = 3 # 가우시안 필터 크기 설정
    gaussian = gaussian_kernel(gaussian_kernel_size, sigma=1)
    # 미분 항들을 가우시안 필터링을 통해 노이즈를 줄인다
    Ix2_blurred = convolve2D(Ix2, gaussian)
    Iy2_blurred = convolve2D(Iy2, gaussian)
    Ixy_blurred = convolve2D(Ixy, gaussian)

    # 헤리스 응답 계산 공식 R=det(M)-k*(trace(M))^2
    detM = Ix2_blurred * Iy2_blurred - Ixy_blurred * Ixy_blurred
    traceM = Ix2_blurred + Iy2_blurred
    R = detM - k * (traceM ** 2)

    # 계산된 R 값을 0~1로 정규화 (선택사항)
    R_min = np.min(R)
    R_max = np.max(R)
    R_normalized = (R - R_min) / (R_max - R_min)

    # 임계값 보다 크고, 주변보다 큰 값만을 코너로 검출
    local_maxima = non_maximum_suppression(R_normalized, threshold)

    return R_normalized, local_maxima


def non_maximum_suppression(response, threshold):
    height, width = response.shape
    local_maxima = np.zeros_like(response, dtype=bool)
    # 이미지의 테두리를 제외한 모둔 픽셀 검
    for y in range(1, height - 1):
        for x in range(1, width - 1):
            # 응답값이 임계값보다 크고 3x3영역에서 자기 자신이 가장 클 경우 해당 위치를 코너로 인식
            if response[y, x] > threshold and response[y, x] == np.max(response[y-1:y+2, x-1:x+2]):
                local_maxima[y, x] = True
    return local_maxima


In [ ]:
# 헤리스 코너 검출 결과 시각화
gray_lena_img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/cv_2024_1/lena.bmp', cv2.IMREAD_GRAYSCALE)

feature_map, keypoints = harris_corner_detector(gray_lena_img)


plt.figure(figsize=(12, 6))
plt.subplot(1,3,1)
plt.imshow(gray_lena_img, cmap='gray')
plt.title("Original Image")

plt.subplot(1, 3, 2)
plt.imshow(feature_map, cmap='gray')
plt.title("Feature Possibility Map")

plt.subplot(1, 3, 3)
plt.imshow(gray_lena_img, cmap='gray')
keypoints_y, keypoints_x = np.where(keypoints)
plt.plot(keypoints_x, keypoints_y, 'r.', markersize=2)
plt.title("Local Maxima Keypoints")
plt.show()
